In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import os
from os import listdir
%matplotlib inline

import base64
import datetime
import io

from textwrap import dedent as d
from scipy import stats
from decimal import Decimal

import plotly.graph_objs as go

In [ ]:
app = dash.Dash()

In [ ]:
# Define graph layout

abs_layout = go.Layout(
    xaxis={'title': 'Voltage [V]'},
    yaxis={'title': 'Current Density [mA/cm^2]'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

In [ ]:
# Format page, reference utilities, markdown explanations

app.layout = html.Div([
    html.H1(html.B('JV Curve')),
    
    html.Div([
        html.Div([dcc.Markdown(d('''
            - Blah''')),
            html.Div([html.B(html.I(
            'Blah'
            ))], 
            style = {
            'fontSize': 20,
            }),
            dcc.Upload(id = 'upload-data',
                       children = html.Button('Upload File'))
                 ],
        style = {
            'maxWidth': 900,
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '20px',
            'borderColor': 'rgb(160, 160, 160)',
            'padding': 20,
            'margin': '0px auto'
        })
    ]),
    
    html.Hr(),
   
    html.Div([
        html.Div([
            html.H3('Blah'),    
            html.Div('Blah'),
            dcc.Graph(
                id='jv-curve',
                figure=go.Figure(),
                selectedData={'points': [], 'range': None}
            )
        ], style={'width': '50%', 'display': 'inline-block'})]),
        
    html.Div([
        html.Table([html.Tr([html.Td(['PCE:']), html.Td(id='PCE')]),
                    html.Tr([html.Td(['VocL:']), html.Td(id='VocL')]),
                    html.Tr([html.Td(['Jsc:']), html.Td(id='Jsc')]),
                    html.Tr([html.Td(['FF:']), html.Td(id='FF')])
                    ], style = {
                                'maxWidth': 900,
                                'borderWidth': '1px',
                                'borderStyle': 'dashed',
                                'borderRadius': '20px',
                                'borderColor': 'rgb(160, 160, 160)',
                                'padding': 20,
                                'margin': '0px auto'
                    })
    ]),
    
    html.Div([
        html.Div([
            html.Label('Enter the desired name of your output file, with .txt at the end:'),
            dcc.Input(
                id='output-filename',
                type='text',
                value='filename.txt'
            ),
    
            html.Div(id='filename'),
         
            html.Button(id='submit-button', n_clicks=0, children='Calculate'),
    
            html.Div(id='output1'),
        ],
            style = {'width': '300',
                    'display': 'inline-block'}
        ),
    ],
    style = {
        'width': '100%',
        'borderWidth': '1px',
        'borderStyle': 'dashed',
        'borderRadius': '20px',
        'borderColor': 'rgb(160, 160, 160)',
        'padding': 10,
        'margin': '0px auto',
        'display': 'inline-block'
        }
    ),
 
    ], style={'textAlign': 'center', 'margin': '10px 30px'})

In [ ]:
# Template for compiling and designing webpage

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

In [ ]:
# Accesses uploaded data and defines dataframe

def dataframe(contents):
    
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    Ldata = pd.read_table(io.StringIO(decoded.decode('utf-8')))
        
    idx_end = Ldata[Ldata.iloc[:,0] == 'Jsc:'].index[0]
    Ldata = Ldata.iloc[:idx_end-1,:]
    Ldata.iloc[:,0] = pd.to_numeric(Ldata.iloc[:,0])
    Ldata.iloc[:,0]
    Ldata = np.array(Ldata)


    Ldata = np.insert(Ldata, 2, -Ldata[:,1], axis=1)

    return Ldata

In [ ]:
# Displays uploaded data

@app.callback(Output('jv-curve', 'figure'),
             [Input('upload-data', 'contents')])
def display_uploaded_data(contents):
    Ldata = dataframe(contents)
    
    a_layout = abs_layout
    
    return go.Figure(
        data=[
            go.Scatter(
                x=Ldata[:,0],
                y=Ldata[:,2],
                mode='lines+markers'
            )
        ],
        layout=a_layout)

In [ ]:
@app.callback(
    [Output('PCE', 'children'),
     Output('VocL', 'children'),
     Output('Jsc', 'children'),
     Output('FF', 'children')],
    [Input('upload-data', 'contents')])
def get_values(contents):
    values = calculate_values(contents)
    PCE = values[0]
    VocL = values[1]
    JscL = values[2]
    FF = values[3]
    
    return PCE, VocL, JscL, FF

In [ ]:
def calculate_values(contents):
    Ldata = dataframe(contents)
    
    JVinterp = interp1d(Ldata[:,0], Ldata[:,2], kind='cubic', bounds_error=False, fill_value='extrapolate')

    JscL = -JVinterp(0)
    VocL = fsolve(JVinterp,.95*max(Ldata[:,0]))
    PPV = fmin(lambda x: x*JVinterp(x),.8*VocL,disp=False)
    PCE = -PPV*JVinterp(PPV)
    FF = PCE/(JscL*VocL)*100
    datas = [PCE, VocL, JscL, FF]
        
    return datas

In [ ]:
# Retrieves outputs for linear mobility

@app.callback(
    Output('output1', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('upload-data', 'contents'),
     State('output-filename', 'value')]
)
def calculate_output(n_clicks, contents, filename):
    
    return_text = ['PCE ', ' VocL ', ' Jsc ', ' FF ']
    total_return = []
    
    values = calculate_values(contents)
    val_one = values[0]
    val_two = values[1]
    val_four = values[3]
    values[0] = val_one[0]
    values[1] = val_two[0]
    values[3] = val_four[0]
    
    for i in range(len(return_text)):
        output = "{:.2E}".format(Decimal(values[i]))
        total_return.append(f"{return_text[i]} = {output}")
    
    if os.path.exists(filename): 
        output_file = np.genfromtxt(filename)
        final_file = np.vstack((output_file, values))
        np.savetxt(filename, final_file, delimiter=" ", fmt="%s", header='PCE, VocL, Jsc, FF')
        
    else:
        np.savetxt(filename, values, delimiter=" ", fmt="%s", header='PCE, VocL, Jsc, FF')

    return total_return

In [ ]:
# Opens browser page to host dashboard

if __name__ == '__main__':
    app.run_server()